In [6]:
import pandas as pd
import os,shutil
from datetime import datetime, timedelta
import datetime

In [7]:
workbook = pd.ExcelFile('latest_sample_pinged_data.xlsx')

In [8]:
data = pd.read_excel(workbook)

In [29]:
data['DNS/CNAME Match']=''

In [30]:
data['DNS/CNAME Match']=ata['DNS']==data['CNAME']

In [31]:
data

,IP,DNS,Tracking Method,Last Scanned Date,Tags,From,IP in CMDB,DNS in CMDB,VM host URL,Ping,CNAME,DNS Match
0,64.89.45.26,rest.na1.netsuite.com,IP,2021-01-19,bo1_external_all|all_external|LightCMS|BU:NSCo...,Cloud,IP NOT in CMDB,DNS NOT in CMDB,NaN,UP,rest.na1.netsuite.com.,False
1,64.89.44.97,suiteconnect.netsuite.com,IP,2021-01-19,bo1_external_all|bo1_lightcms_external_all|all...,Cloud,IP NOT in CMDB,DNS NOT in CMDB,NaN,UP,suiteconnect.netsuite.com.,False
2,64.89.44.98,None,IP,2021-01-19,bo1_external_all|bo1_lightcms_external_all|all...,Cloud,IP NOT in CMDB,DNS NOT in CMDB,NaN,UP,None,True
3,64.89.44.99,None,IP,2021-01-19,bo1_external_all|bo1_lightcms_external_all|all...,Cloud,IP NOT in CMDB,DNS NOT in CMDB,NaN,UP,None,True
4,64.89.44.101,None,IP,2021-01-19,bo1_external_all|bo1_lightcms_external_all|all...,Cloud,IP NOT in CMDB,DNS NOT in CMDB,NaN,UP,None,True
...,...,...,...,...,...,...,...,...,...,...,...,...
57870,172.17.42.155,se4-kms01,AGENT,2021-01-19,NSIT Cloud Agent|Cloud Agent,PCP,IP NOT in CMDB,DNS NOT in CMDB,NaN,UP,sc8-dnsorcl01.corp.netledger.com.,False
57871,172.17.42.40,sc8-mailmgmt01,AGENT,2021-01-19,NSIT Cloud Agent|Cloud Agent,PCP,IP NOT in CMDB,DNS NOT in CMDB,NaN,UP,sc8-mailmgmt01.corp.netledger.com.,False
57872,172.17.42.134,sc8-scanner,AGENT,2021-01-19,NSIT Cloud Agent|Cloud Agent,PCP,IP NOT in CMDB,DNS NOT in CMDB,NaN,UP,SC8-scanner.corp.netledger.com.,False
57873,192.168.201.34,radchikov.brontolabs.local,AGENT,2021-01-19,Bronto Cloud Agent|Cloud Agent,PCP,IP NOT in CMDB,DNS NOT in CMDB,NaN,DOWN,None,False


In [ ]:
#extracts the filtered data to its own variable
def sheet(source,column, string):
    x = source[source[column].str.contains(string,na=False)]
    x = x.sort_values('IP')
    return x
#removes the filtered data from the main variable
def sheet_x(source,column, string):
    x = source[~source[column].str.contains(string,na=False)]
    x = x.sort_values('IP')
    return x

In [ ]:
#change last scanned date to_date
data['Last Scanned Date']=pd.to_datetime(data['Last Scanned Date'], format= '%Y-%m-%d').dt.date

In [ ]:
#getting duplicates by date
duplicates = data[data.sort_values('Last Scanned Date').duplicated('IP', keep='first')]

In [ ]:
#dropping duplicates keeping last/recent instance
data = data.sort_values('Last Scanned Date').drop_duplicates('IP', keep='last')

In [ ]:
#older than 30days from last scanned date
old = data[data['Last Scanned Date'] < datetime.date(2021,1,19)- pd.to_timedelta("30day") ].sort_values('Last Scanned Date' ,ascending=False)

In [ ]:
#30 days from last scanned date
data = data[data['Last Scanned Date'] > datetime.date(2021,1,19)- pd.to_timedelta("30day") ].sort_values('Last Scanned Date' ,ascending=False)

In [ ]:
len(data)+len(old)+len(duplicates)

In [ ]:
57873-50226

In [ ]:
#Tags
agent = sheet(data,'Tracking Method','AGENT')
main = sheet_x(data,'Tracking Method','AGENT')

In [ ]:
#Tags
netops = sheet(main,'Tags','NetOps')
main = sheet_x(main,'Tags','NetOps')

In [ ]:
#Tags
nscore = sheet(main,'Tags','NSCore')
main = sheet_x(main,'Tags','NSCore')

In [ ]:
#Tags
bronto = sheet(main,'Tags','Bronto')
main = sheet_x(main,'Tags','Bronto')

In [ ]:
#Tags
openair = sheet(main,'Tags','OpenAir')
main = sheet_x(main,'Tags','OpenAir')

In [ ]:
#Tags
payroll = sheet(main,'Tags','Payroll')
main = sheet_x(main,'Tags','Payroll')

In [ ]:
#Tags
devops = sheet(main,'Tags','DevOps')
main = sheet_x(main,'Tags','DevOps')

In [ ]:
#Tags
pub = sheet(main,'Tags','Internet Facing Assets')
main = sheet_x(main,'Tags','Internet Facing Assets')

In [ ]:
#Tags
nspos = sheet(main,'Tags','NSPOS')
main = sheet_x(main,'Tags','NSPOS')

In [ ]:
#Tags
test_asset = sheet(main,'Tags','Test Asset')
main = sheet_x(main,'Tags','Test Asset')

In [ ]:
#Tags
legacy = sheet(main,'Tags','LegacyCorp')
main = sheet_x(main,'Tags','LegacyCorp')

In [ ]:
bu_frames = [main,agent,netops,nscore,bronto,openair,payroll,devops,pub,nspos,test_asset,legacy,duplicates,old]
bu_titles = ['main','agent','netops','nscore','bronto','openair','payroll','devops','pub','nspos','test_asset','legacy','duplicates','old']

In [ ]:
def writer(bu_frames,filter_frame,bu_titles,filter_title,types):
        with pd.ExcelWriter(bu_title+'/'+types+bu_title+'.xlsx') as writer:
            print('writing')
            for x,y in zip(filter_frame,filter_title):
                if len(x) > 1:
                    x.to_excel(writer,sheet_name=y ,index=False)
                    print(y, 'created')
                else:
                    print(y,'empty')
def filters (bu_frames, bu_titles, types):
    print('Working on',bu_titles)
    print('Sorting "Duplicate IP"')
    duplicate = bu_frames[bu_frames.duplicated(subset=['IP'],keep='last')]
    main = bu_frames[~bu_frames.duplicated(subset=['IP'],keep='last')]
    print('Sorting "Available"')
    available = sheet(main,'DNS','available')
    main = sheet_x(main,'DNS','available')
    available = available.append(sheet(main,'CNAME','AVAILABLE'))
    main = sheet_x(main,'CNAME','AVAILABLE')
    print('Sorting "VIP DNS"')
    vip = sheet(main,'DNS','vip')
    main = sheet_x(main,'DNS','vip')
    print('Sorting "Internet Facing Assets"')
    pub = sheet(main,'Tags','Internet Facing Assets')
    main = sheet_x(main,'Tags','Internet Facing Assets')
    print('Sorting "Load Balancers"')
    lb = sheet(main,'DNS','-mip|-vlb|-plb')
    main = sheet_x(main,'DNS','-mip|-vlb|-plb')
    #Tags
    lb = sheet(main,'Tags','loadbalancer')
    main = sheet_x(main,'Tags','loadbalancer')
    print('Sorting "Agent"')
    agent = sheet(main,'Tracking Method','AGENT')
    main = sheet_x(main,'Tracking Method','AGENT')
    print('Sorting "Interface"')
    interface = sheet(main,'DNS','interface')
    main = sheet_x(main,'DNS','interface')
    print('Sorting "links"')
    link = sheet(main,'DNS','-link')
    main = sheet_x(main,'DNS','-link')
    print('Sorting "None"')
    blank = sheet(main,'DNS','None|none')
    main = sheet_x(main,'DNS','None|none')
    print('Sorting "DevOps"')
    devops = sheet(main,'Tags','BU:DevOps')
    main = sheet_x(main,'Tags','BU:DevOps')
    print('Sorting "Duplicate DNS"')
    duplicate = duplicate.append(main[main.duplicated(subset=['DNS'],keep='last')])
    main = main[~main.duplicated(subset=['DNS'],keep='first')]
    print('Sorting "LS"')
    ls = sheet(main,'DNS','-ls')
    main = sheet_x(main,'DNS','-ls')
    print('Sorting "CY"')
    cy = sheet(main,'DNS','cy')
    main = sheet_x(main,'DNS','cy')
    print('Sorting "bo1"')
    bo1 = sheet(main,'DNS','BO1|bo1')
    main = sheet_x(main,'DNS','BO1|bo1')
    #Tags
    bo1 = bo1.append(sheet(main,'Tags','BO1|bo1'))
    main = sheet_x(main,'Tags','BO1|bo1')
    print('Sorting "Bronto"')
    bronto = sheet(main,'DNS','Bronto|bronto')
    main = sheet_x(main,'DNS','Bronto|bronto')
    #tags
    bronto = bronto.append(sheet(main,'Tags','Bronto|bronto'))
    main = sheet_x(main,'Tags','Bronto|bronto')
    print('Sorting "OpenAir"')
    openair = sheet(main,'Tags','openair|OpenAir')
    main = sheet_x(main,'Tags','openair|OpenAir')
    print('Sorting "OCI"')
    oci = sheet(main,'DNS','OCI|oci')
    main = sheet_x(main,'DNS','OCI|oci')
    #tags
    oci = oci.append(sheet(main,'Tags','OCI|oci'))
    main = sheet_x(main,'Tags','OCI|oci')
    print('Sorting "Payroll"')
    payroll = sheet(main,'Tags','payroll|Payroll')
    main = sheet_x(main,'Tags','payroll|Payroll')
    print('Sorting "VPN"')
    vpn = sheet(main,'DNS','vpn|ovpn')
    main = sheet_x(main,'DNS','vpn|ovpn')
    #tags
    vpn = vpn.append(sheet(main,'Tags','vpn|ovpn'))
    main = sheet_x(main,'Tags','vpn|ovpn')
    print('Sorting "Console"')
    console = sheet(main,'DNS','console')
    main = sheet_x(main,'DNS','console')
    #tags
    console = console.append(sheet(main,'Tags','console'))
    main = sheet_x(main,'Tags','console')
    print('Sorting "NoUse"')
    nouse = sheet(main,'DNS','nouse')
    main = sheet_x(main,'DNS','nouse')
    print('Sorting "VM"')
    vm = sheet(main,'DNS','snap|\.f\.|\.m\.')
    main = sheet_x(main,'DNS','snap|\.f\.|\.m\.')
    print('Sorting "PDU"')
    pdu = sheet(main,'DNS','pdu-')
    main = sheet_x(main,'DNS','pdu-')
    #IP address filter
    pdu = pdu.append(sheet(main,'IP','^10\..*\..*\.24[0-8]'))
    main = sheet_x(main,'IP','^10\..*\..*\.24[0-8]')
    print('Finished sorting filters')
    filter_frame = [main,lb,devops,link, blank, duplicate, agent, ls, cy, pdu, pub, bo1, bronto, openair, oci, payroll, console, vpn, nouse, vm,interface, available,vip]
    filter_title = ['main','Load Balancer','DevOps','Link', 'None', 'Duplicate', 'Agent', 'LS', 'CY', 'PDU', 'Internet Facing Asset', 'BO1', 'Bronto', 'OpenAir', 'OCI'
                    , 'Payroll', 'Console', 'VPN', 'NoUse', 'VM','Interface', 'Available', 'VIP_NAT']
    writer(bu_frame,filter_frame,bu_title,filter_title,types)
    
def cmdb(x):
    ins = x[(x['IP in CMDB']=='IP in CMDB') | (x['DNS in CMDB'] == 'DNS in CMDB')]
    out = x[(x['IP in CMDB'] =='IP NOT in CMDB') & (x['DNS in CMDB'] == 'DNS NOT in CMDB')]
    return ins, out

In [ ]:

        
for bu_frame,bu_title in zip(bu_frames, bu_titles):
    if len(bu_frame) > 1:
        if os.path.exists(bu_title):
            shutil.rmtree(bu_title)
        os.mkdir(bu_title)
        
        ins, out = cmdb(bu_frame)
        if len(ins)>1: 
            filters (ins,bu_title,"IN_CMDB_")
        else:
            print("EMPTY DataFrame")

        if len(out)>1:
            filters (out,bu_title,"OUT_CMDB_")
        else:
            print("EMPTY DataFrame")
            
    else:
        pass
    
    
        
# with pd.ExcelWriter('test.xlsx') as writer:
#     main.to_excel(writer,sheet_name='main', index=False)
#     agent.to_excel(writer,sheet_name='AGENT', index=False)
#     netops.to_excel(writer,sheet_name='netops', index=False)
#     nscore.to_excel(writer,sheet_name='nscore', index=False)
#     bronto.to_excel(writer,sheet_name='bronto', index=False)
#     openair.to_excel(writer,sheet_name='openair', index=False)
#     payroll.to_excel(writer,sheet_name='payroll', index=False)
#     devops.to_excel(writer,sheet_name='devops', index=False)
#     pub.to_excel(writer,sheet_name='Internet Facing Assets', index=False)
#     nspos.to_excel(writer,sheet_name='NSPOS', index=False)
#     test_asset.to_excel(writer,sheet_name='Test Asset', index=False)
#     legacy.to_excel(writer,sheet_name='LegacyCorp', index=False)
    
    
    
    
    
    
    
    
    